In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
#client = OpenAI()

In [ ]:
client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [4]:
chat_completion = client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    reasoning_effort="low",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "hello world"
        }
    ]
)

In [5]:
# Using OpenAI's API instead of google one ==> API  key must be set on .env file
#chat_completion = client.chat.completions.create(
#    model="gpt-3.5-turbo",
#    messages=[{"role": "user", "content": "Hello world"}]
#)

In [6]:
chat_completion.choices[0].message.content

'Hello! How can I help you today?'

In [7]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gemini-2.5-flash-preview-04-17",
            reasoning_effort="low",
            messages=self.messages)
        return completion.choices[0].message.content
    

In [8]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [9]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [10]:
abot = Agent(prompt)

In [11]:
result = abot("How much does a toy poodle weigh?")
print(result)

Action: average_dog_weight: Toy Poodle
PAUSE


In [12]:
result = average_dog_weight("Toy Poodle")

In [13]:
result

'a toy poodles average weight is 7 lbs'

In [14]:
next_prompt = "Observation: {}".format(result)

In [15]:
abot(next_prompt)

"Answer: A toy poodle's average weight is 7 lbs"

In [16]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [17]:
abot = Agent(prompt)

In [18]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: The user wants the combined weight of two dogs: a border collie and a scottish terrier. I need to get the weight for each breed and then sum them up. I have already obtained the weight for the Border Collie. Now I need to get the weight for the Scottish Terrier using the `average_dog_weight` action.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [19]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [20]:
abot(next_prompt)

"Thought: I have the weight for the Border Collie (37 lbs). I need to get the weight for the Scottish Terrier to calculate the combined weight. I should use the `average_dog_weight` action for 'Scottish Terrier'.\nAction: average_dog_weight: Scottish Terrier\nPAUSE"

In [21]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [22]:
abot(next_prompt)

'Thought: I have the average weight for both dogs: Border Collie (37 lbs) and Scottish Terrier (20 lbs). I need to calculate their combined weight by adding these two values together. I should use the `calculate` action to perform the addition.\nAction: calculate: 37 + 20\nPAUSE'

In [23]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [24]:
abot(next_prompt)

'Thought: I have successfully retrieved the average weights for both dogs and calculated their sum. The combined weight is 57 lbs. I can now provide the answer to the user.\nAnswer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

# Add Loop

In [25]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/c5/nf1tlttx5vb4bzl7f_238wdc0000gr/T/ipykernel_40540/2389047760.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action


In [26]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier and then sum them. I will start by getting the weight of the Border Collie.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: I have the average weight for the Border Collie. Now I need to find the average weight for the Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have the average weights for both dogs. Now I need to calculate their combined weight by adding the two weights together.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Thought: I have calculated the combined average weight of the two dogs. I can now provide the answer.
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.
